## Uncertainty estimation for regression
We would demonstrate how to estimate the uncertainty for a regression task. In this case we treat uncertainty as a standard deviation for test data points.
As an example dataset we take the kinemtic movement data from UCI database and would estimate the uncertainty prediction with log likelihood metric

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import r2_score 

from alpaca.ue import MCDUE
from alpaca.utils.datasets.builder import build_dataset
from alpaca.utils.ue_metrics import ndcg, uq_ll
from alpaca.ue.masks import BasicBernoulliMask, DecorrelationMask, LeverageScoreMask
from alpaca.utils import model_builder
import alpaca.nn as ann

## Prepare the dataset
The alpaca library has a few regression dataset provided (these datasets often used in the related scientific papers)

In [2]:
dataset = build_dataset('kin8nm', val_split=1_000)
x_train, y_train = dataset.dataset('train')
x_val, y_val = dataset.dataset('val')
x_train.shape, y_val.shape
train_ds = TensorDataset(torch.FloatTensor(x_train), torch.FloatTensor(y_train))
val_ds = TensorDataset(torch.FloatTensor(x_val), torch.FloatTensor(y_val))
train_loader = DataLoader(train_ds, batch_size=512)
val_loader = DataLoader(val_ds, batch_size=512)

## Let's build the simple model
We'll replace common nn.Dropout layer with ann.Dropout from alpaca.
Alpaca version allow to switch on the dropout during inference without worrying other "training" layers, like batch norm.

In [3]:
class MLP(nn.Module):
    def __init__(self, input_size, base_size=64, dropout_rate=0., dropout_mask=None):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 4*base_size),
            nn.CELU(),

            nn.Linear(4*base_size, 2*base_size),
            ann.Dropout(dropout_rate, dropout_mask),
            nn.CELU(),

            nn.Linear(2*base_size, 1*base_size),
            ann.Dropout(dropout_rate, dropout_mask),
            nn.CELU(),

            nn.Linear(base_size, 1)
        )

    def forward(self, x):
        return self.net(x)

In [4]:
# Train model
model = MLP(input_size=8, dropout_rate=0.1, dropout_mask=BasicBernoulliMask)

## Train the model

In [5]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

model.train()
for epochs in range(100):
    for x_batch, y_batch in train_loader: # Train for one epoch
        predictions = model(x_batch)
        loss = criterion(predictions, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
print('Train loss on last batch', loss.item())

Train loss on last batch 0.000422369601437822


In [6]:
# Check model effectiveness 
model.eval()
x_batch, y_batch = next(iter(val_loader))
predictions = model(x_batch).detach().cpu().numpy()
print('R2:', r2_score(predictions, y_batch))


R2: 0.9217043442551243


## Estimate uncertainty
We compare the log likelihood for constant prediction and monte-carlo uncertainty estimation

In [7]:
# Baseline
const_std = np.std(y_val)
errors = np.abs(predictions - y_batch.reshape((-1)).numpy())
score = uq_ll(errors, np.ones_like(errors) * const_std)
print("Quality score for const std is ", score)


Quality score for const std is  0.27723035


In [8]:
model.train()
estimator = MCDUE(model, nn_runs=100)

In [9]:
predictions, estimations = estimator(x_batch)

Uncertainty estimation with MCDUE_regression approach: 100%|██████████| 100/100 [00:00<00:00, 2998.07it/s]


In [10]:
errors = np.abs(predictions - y_batch.reshape((-1)).numpy())
score = uq_ll(np.array(errors), predictions)
print("Quality score for monte-carlo dropout is ", score)


Quality score for monte-carlo dropout is  0.37827393
